In [89]:
import re
import os
import folium
import glob
import logging
import numpy as np
from geopy.geocoders import Nominatim
from folium import plugins
import matplotlib.pyplot as plt
from matplotlib import colors

In [103]:
placelist = ['Inca', 'Calvia', 'Palma de Mallorca', 'Felanitx', 'Pollensa', 'Colonia de Sant Pere',
             'Alcudia', 'Andratx', 'Soller', 'Deia', 'Valldemossa', 'Fornalutx', 'Lluc', 
             'Llucmajor', 'Cala Millor', 'Orient', 'Bunyola', 'Arta', 'Capdepera', 'Randa', 'Estellencs',
             'Establiments', 'Banyalbufar', 'Esporles', 'Port de Pollensa', 'Portocristo',
             'Son Servera', 'Santa Margalida', 'Montuiri', 'Santa María del Cami', 'Can Picafort',
             'Petra', 'Son Serra de Marina', 'Campanet', "Caimari", 'Selva', 'Mancor de la Vall',
             'Lloseta', 'Alaro', 'Puigpunyent', 'Galilea', 'Sant Elm', "Port d'Andratx",
             'Peguera', 'Santa Ponça', 'es Molinar', 'Ariany', 'Santa Margalida', 'Porreres',
             'Santanyi', 'Colonia de Sant Jordi', 'sa Rapita', 'Selva', 'Moscari, Raiguer', 'Biniamar']

In [104]:
puiglist = ["Puig d'Alaro", "Puig de l'Ofre", "Moleta de s'Esclop", "Puig de Galatzo", "Puig des Porrassar",
            "Fita des Ram", "es Penyal Gran", "Es Caragoli", "Puig des Teix", "Puig de s'Aritjar",
            "Puig d'Alfabia", "Puig des Tossals Verds", "Puig de Massanella", "Penyal d'Honor", 
            "Puig d'Alaro", "Sant Salvador", "Puig Tomir", "Puig de Maria",
            "Na Blanca", "Es Caragoli", 'Puig de Randa', 'Puig des Revells',
            "Na Penyal", "Puig de Planicia", "Talaia d'Alcudia", "Puig des Romani",
            "Penyalet des Clot des Cero", "Sa Talaeita", "Puig des Migdia, Arta", "sa Talaia Freda",
            "Sa Moleta"]

In [111]:
lighthouses = ["Far de Capdepera", "Far del Cap de Formentor"]

The list of already geolocalised place is stored somewhere, so we don't have to look for them every time we run the code. Except if we want to start the list from scratch. The file will have 3 columns: name of the place, latitude and longitude. 

In [112]:
listname = './place_list.txt'
puiglistname = './puig_list.txt'
lighthousename = "./lighthouse_list.txt"

We start by reading the place names written inside this file, if it exists.

In [113]:
saved_places, saved_puig, saved_lighthouse = [], [], []
if os.path.exists(listname):
    with open(listname, 'r') as f:
        for lines in f:
            saved_places.append(lines.split('|')[0])
if os.path.exists(puiglistname):
    with open(puiglistname, 'r') as f:
        for lines in f:
            saved_puig.append(lines.split('|')[0])   
if os.path.exists(lighthousename):
    with open(lighthousename, 'r') as f:
        for lines in f:
            saved_lighthouse.append(lines.split('|')[0])
else:
    open(lighthousename, 'a').close()

Now we can update the list of with the new list of places we want to add on the map.

In [114]:
geolocator = Nominatim()
with open(listname, 'a') as f:
    for places in placelist:
        if places in saved_places:
            continue
            #print 'already done'
        else:
            print(places)
            location = geolocator.geocode({'city' : places, "country": "Spain"}, timeout=10)
            if location:
                print location.raw
                f.write("%s|%.7f|%.7f\n" % (places, location.latitude, location.longitude))

Moscari, Raiguer


In [115]:
with open(puiglistname, 'a') as f:
    for puig in puiglist:
        if puig in saved_puig:
            continue
            print 'already done'
        else:
            print puig
            location = geolocator.geocode(puig, timeout=10)
            if location:
                f.write("%s|%.7f|%.7f\n" % (puig, location.latitude, location.longitude))

In [116]:
with open(lighthousename, 'a') as f:
    for lighthouse in lighthouses:
        if lighthouse in saved_lighthouse:
            continue
            print 'already done'
        else:
            print lighthouse
            location = geolocator.geocode(lighthouse, timeout=10)
            if location:
                f.write("%s|%.7f|%.7f\n" % (lighthouse, location.latitude, location.longitude))

Far de Capdepera
Far del Cap de Formentor


# Create the map

In [121]:
location = geolocator.geocode("Lloret de Vistalegre, Mallorca, Spain", timeout=10)
map_run = folium.Map(location=[location.latitude, location.longitude], 
                     zoom_start=10)

# Create custom icon for mountain
iconurl_mountain = 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/5a/Mountain_Icon.svg/128px-Mountain_Icon.svg.png'
iconurl_lighthouse = 'https://github.com/ctroupin/trail-running-maps/raw/master/images/lighthouse-icon.png'

with open(listname, 'r') as f:
    for lines in f:
        latlon = (lines.split('|')[1:])
        folium.CircleMarker(latlon, radius=300, color='blue', 
                            fill_color='blue', fill_opacity=0.8, popup=None).add_to(map_run)
        
with open(puiglistname, 'r') as f:
    for lines in f:
        latlon = (lines.split('|')[1:])
        
        iconmountain = folium.features.CustomIcon(iconurl_mountain, icon_size=(20, 20))
        folium.Marker(latlon, 
                      popup=lines.split('|')[0], icon=iconmountain).add_to(map_run)

with open(lighthousename, 'r') as f:
    for lines in f:
        latlon = (lines.split('|')[1:])
        iconlighthouse = folium.features.CustomIcon(iconurl_lighthouse, icon_size=(20, 20))
        folium.Marker(latlon, 
                      popup=lines.split('|')[0], icon=iconlighthouse).add_to(map_run)

In [122]:
map_run

In [71]:
outputdir = '/home/ctroupin/public_html/LeafletMaps'
mapname = 'test_location_points.html'
if not(os.path.exists(outputdir)):
    os.makedirs(outputdir)
    logging.info('Creating directory %s' %(outputdir))
map_run.save(os.path.join(outputdir, mapname))